In [18]:
!pip install langchain -q

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains.summarize import load_summarize_chain
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import pipeline
import torch
import base64
import PyPDF2
import requests
import string
from io import BytesIO
import re



In [2]:
import PyPDF2
import requests
import string
from io import BytesIO

# URL of the PDF
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"

# Download the PDF content using requests
response = requests.get(pdf_url)
pdf_content = response.content

# Create a PDF reader
pdf_reader = PyPDF2.PdfReader(BytesIO(pdf_content))

# Extract text from pages 2 to 5 (index 1 to 4)
start_page = 1  # Page index to start from
end_page = min(start_page +4, len(pdf_reader.pages))  # End at page index 5 or last page, whichever comes first

extracted_text = ""
for page_number in range(start_page, end_page):
    page = pdf_reader.pages[page_number]
    extracted_text += page.extract_text()

# Clean up the extracted text
cleaned_text = " ".join(extracted_text.split())  # Remove extra spaces

# Print the cleaned extracted text
print(cleaned_text)
print(len(cleaned_text))


1 Introduction Recurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [ 35,2,5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15]. Recurrent models typically factor computation along the symbol positions of the input and output sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden states ht, as a function of the previous hidden state ht−1and the input for position t. This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples. Recent work has achieved significant improvements in computational efficiency through factor

In [3]:
#model and tokenizer loading
checkpoint = "LaMini-Flan-T5-248M"
tokenizer = T5Tokenizer.from_pretrained(checkpoint)
base_model = T5ForConditionalGeneration.from_pretrained(checkpoint, device_map='auto', torch_dtype=torch.float32)


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [6]:

pipe_sum = pipeline(
    'summarization',
    model = base_model,
    tokenizer = tokenizer,
    max_length = 500, 
    min_length = 50)

In [7]:
%time
result = pipe_sum(cleaned_text)
result = result[0]['summary_text']
result


Token indices sequence length is longer than the specified maximum sequence length for this model (2760 > 512). Running this sequence through the model will result in indexing errors


'The Transformer is a model architecture based on stacked self-attention and point-wise, fully connected layers for both the encoder and decoder. The Transformer uses multi-head attention to draw global dependencies between input and output, allowing for more parallelization and reaching a new state of the art in translation quality. Multi-head Attention is used to generate a sequence of hidden states, reducing sequential computation and improving model performance. It is also used in conjunction with a recurrent network to reduce computational cost and improve model performance in various tasks. In addition, the Transformer employs h=8 parallel attention layers, where each layer produces outputs of dimension dmodel = 512, and the output is computed as a weighted sum 3Scaled Dot-Product Attention. This allows the model to jointly attend to information from different representation subspaces at different positions, resulting in large dot products.'

# 2

In [ ]:
%%time
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"

# Download the PDF content using requests
response = requests.get(pdf_url)
pdf_content = response.content

# Create a PDF reader
pdf_reader = PyPDF2.PdfReader(BytesIO(pdf_content))

# Extract text from pages 2 to 5 (index 1 to 4)
start_page = 1  # Page index to start from
end_page = min(start_page +4, len(pdf_reader.pages))  # End at page index 5 or last page, whichever comes first

extracted_text = ""
for page_number in range(start_page, end_page):
    page = pdf_reader.pages[page_number]
    extracted_text += page.extract_text()

# Clean up the extracted text
cleaned_text = " ".join(extracted_text.split())  # Remove extra spaces

pipe_sum = pipeline('summarization',model = base_model,tokenizer = tokenizer,max_length = 1000, min_length = 50)
result = pipe_sum(cleaned_text)
result = result[0]['summary_text']
print(result)
print(len)

In [22]:
import requests
import PyPDF2
from io import BytesIO
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Define the PDF URL
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"

# Download the PDF content using requests
response = requests.get(pdf_url)
pdf_content = response.content

# Create a PDF reader
pdf_reader = PyPDF2.PdfReader(BytesIO(pdf_content))

# Extract text from pages 2 to 5 (index 1 to 4)
start_page = 1  # Page index to start from
end_page = min(start_page + 4, len(pdf_reader.pages))  # End at page index 5 or last page, whichever comes first

extracted_text = ""
for page_number in range(start_page, end_page):
    page = pdf_reader.pages[page_number]
    extracted_text += page.extract_text()

# Split the extracted text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
text_chunks = text_splitter.split_text(extracted_text)

# Combine the chunks into final text
final_text = "".join(text_chunks)

# Print or return the final text
print("final_text")
print(final_text)
print("\n\\n\n")
pipe_sum = pipeline('summarization',model = base_model,tokenizer = tokenizer,max_length = 1000, min_length = 50)
result = pipe_sum(final_text)
result = result[0]['summary_text']
print("result")
print(result)
print(len)

final_text
1 Introduction
Recurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networksin particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [ 35,2,5]. Numerousefforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and outputsequences. Aligning the positions to steps in computation time, they generate a sequence of hiddenstates ht, as a function of the previous hidden state ht−1and the input for position t. This inherentlysequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achievedsignificant improvements in computa

In [23]:
print(len(result))

872


In [30]:
%%time
import requests
import PyPDF2
from io import BytesIO
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Define the PDF URL
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"

# Download the PDF content using requests
response = requests.get(pdf_url)
pdf_content = response.content

# Create a PDF reader
pdf_reader = PyPDF2.PdfReader(BytesIO(pdf_content))

# Extract text from pages 2 to 5 (index 1 to 4)
start_page = 1  # Page index to start from
end_page = min(start_page + 4, len(pdf_reader.pages))  # End at page index 5 or last page, whichever comes first

extracted_text = ""
for page_number in range(start_page, end_page):
    page = pdf_reader.pages[page_number]
    extracted_text += page.extract_text()

# Split the extracted text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
text_chunks = text_splitter.split_text(extracted_text)

# Combine the chunks into final text
final_text = "".join(text_chunks)
final_text = re.sub(r'[^a-zA-Z0-9\s]', '', final_text)
final_text   = re.sub(r'\S*@\S*\s?', '', final_text)
final_text= final_text.rstrip()
# Print or return the final text
print("final_text")
print(final_text)



final_text
1 Introduction
Recurrent neural networks long shortterm memory  13 and gated recurrent  7 neural networks
in particular have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation  3525 Numerous
efforts have since continued to push the boundaries of recurrent language models and encoderdecoder
architectures 38 24 15architectures 38 24 15
Recurrent models typically factor computation along the symbol positions of the input and output
sequences Aligning the positions to steps in computation time they generate a sequence of hidden
states ht as a function of the previous hidden state ht1and the input for position t This inherently
sequential nature precludes parallelization within training examples which becomes critical at longersequence lengths as memory constraints limit batching across examples Recent work has achieved
significant improvements in computational efficiency through 

In [33]:
%%time 
pipe_sum = pipeline('summarization',model = base_model,tokenizer = tokenizer,max_length = 512, min_length = 50)
result = pipe_sum(final_text)
result = result[0]['summary_text']
print("result")
print(result)
print(len)

result
The Transformer is a neural network architecture that uses stacked selfattention and pointwise fully connected layers for both the encoder and decoder layers. It is the first transduction model relying entirely on attention mechanisms to draw global dependencies between input and output. The Transformer uses multihead attention and multi-head attention to reduce sequential computation and improve model performance. Multihead attention is used to generate a sequence of hidden states and outputs. The output is computed as a weighted sum 3Scaled DotProduct Attention and MultiHead Attention. The focus is on reducing the number of operations required to relate signals from two arbitrary input or output positions. The attention function can be described as mapping a query and a set of keyvalue pairs to an output where the query keys and values are all vectors. This allows the model to jointly attend to information from different representation subspaces at different positions. To coun

'The Transformer is a model architecture based on stacked self-attention and point-wise, fully connected layers for both the encoder and decoder. The Transformer uses multi-head attention to draw global dependencies between input and output, allowing for more parallelization and reaching a new state of the art in translation quality. Multi-head Attention is used to generate a sequence of hidden states, reducing sequential computation and improving model performance. It is also used in conjunction with a recurrent network to reduce computational cost and improve model performance in various tasks. In addition, the Transformer employs h=8 parallel attention layers, where each layer produces outputs of dimension dmodel = 512, and the output is computed as a weighted sum 3Scaled Dot-Product Attention. This allows the model to jointly attend to information from different representation subspaces at different positions, resulting in large dot products.'

The Transformer is a neural network architecture that uses multi-head attention to generate outputs of different positions in the input sequence. It is the first transduction model that relies entirely on attention mechanisms to draw global dependencies between input and output. The Transformer uses stacked self-attention and point-wise, fully connected layers for both the encoder and decoder layers, and a multi-head attention function to achieve more parallelization and improve model performance. Multi-Head Attention allows the model to jointly attend to information from different representation subspaces at different positions, reducing the computational cost and improving model performance in cases of sequential computation. The transformer uses learned embeddings and softmax functions to convert inputtokens and output tokens to vectors of dimension dmodel.

The Transformer is a neural network architecture that uses stacked selfattention and pointwise fully connected layers for both the encoder and decoder layers. It is the first transduction model relying entirely on attention mechanisms to draw global dependencies between input and output. The Transformer uses multihead attention and multi-head attention to reduce sequential computation and improve model performance. Multihead attention is used to generate a sequence of hidden states and outputs. The output is computed as a weighted sum 3Scaled DotProduct Attention and MultiHead Attention. The focus is on reducing the number of operations required to relate signals from two arbitrary input or output positions. The attention function can be described as mapping a query and a set of keyvalue pairs to an output where the query keys and values are all vectors. This allows the model to jointly attend to information from different representation subspaces at different positions. To counteract this effect, we scale the dot products by1dk 322 Multihead Attention. This involves dividing the input by the weight assigned to each value and applying a softmax function to obtain the weights on the values. This prevents leftward information flow and preserves the autoregressive property. We implement this inside of scaled dotproduct attention by masking out setting to all values in the input of the softmax which corresponds to illegal connections. We share the same weight matrix between the two embedding layers and the presoftmax linear transformation similar to 30. In the embeddding layers, we multiply those weights bydmodel 5.